In [2]:
import synthetic_signal as synth
import numpy as np
import distance as distance
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import sliding_window_view
from scipy.spatial.distance import cdist

In [2]:
import numpy as np
import distance as distance
   
class ValmodPairMotif(object):

    def __init__(self,n_patterns:int, min_wlen:int, max_wlen:int, p:int, distance_name:str, distance_params =dict())-> None:
        """Initialization

            Args:
                n_patterns (int): Number of neighbors
                min_wlen (int): Minimum window length
                max_wlen (int): Maximum window length
                p(int): minimal number of distances computed in any cases
                distance_name (str): name of the distance
                distance_params (dict, optional): additional distance parameters. Defaults to dict().
        """
        self.n_patterns = n_patterns
        self.min_wlen = min_wlen
        self.max_wlen = max_wlen
        self.p = p 
        self.distance_name = distance_name
        self.distance_params = distance_params

    def CompLB(self,idx:int,i:int)->np.ndarray:
        """Compute the LowerBound of the distance between Ti,l+1 and Tj,l+1 for all j
            Args:
                idx: window length index
                i (int): considered subsequence
            returns: 
                np.ndarray: lower bounds array
        """
        wlen=self.wlens_[idx]
        next_nDP=self.n-wlen
        next_non_overlap_mask=np.arange(max(0,i-wlen), min(next_nDP,i+wlen+1))
        #we compute qi,j for all j
        q=(self.distance_[idx].dot_product_/wlen -self.distance_[idx].means_[i]*self.distance_[idx].means_)/(self.distance_[idx].stds_[i]*self.distance_[idx].stds_)
        #we truncate it because for the next wlen we have less possible subsequences  
        q=q[:next_nDP]
        
        LB=np.zeros(next_nDP)
        mask=q<=0
        LB[mask]=np.sqrt(wlen)*(self.distance_[idx].stds_[:next_nDP][mask]/self.distance_[idx+1].stds_[mask])
        LB[~mask]=np.sqrt(wlen*(1-q[~mask]**2))*(self.distance_[idx].stds_[:next_nDP][~mask]/self.distance_[idx+1].stds_[~mask])

        LB[next_non_overlap_mask]=np.inf

        return LB
    
    def ComputeMatrixProfile(self,idx:int)-> tuple:
        """Compute the Matrix Profile and the LowerBound for the length corresponding to idx
            Args:
                idx(int): window length index
            Returns: 
                MP(np.ndarray): MatrixProfile
                IP(np.ndarray: Index Profile
                listDP(list of np.ndarray): list containing for each i the successive informations:
                    -the indexes of the p minimum Dij
                    -the corresponding distances
                    -the corresponding LB
                    -the corresponding dot_products
                """
        wlen=self.wlens_[idx]
        nDP=self.n-wlen+1
        MP=np.zeros(nDP)
        IP=np.zeros(nDP).astype(int)
        listDP=[]

        line=self.distance_[idx].first_line(0)
        non_overlap_mask = np.arange(0, min(nDP,wlen))
        line[non_overlap_mask] = np.inf
        MP[0]=np.min(line)
        IP[0]=np.argmin(line)
        #we don't need the lower bound for the last index 
        if idx+1< self.wlens_.shape[0]:
            LB=self.CompLB(idx,0)
            idx_sort=np.argsort(LB)[:self.p]
            trunc_dist=line[idx_sort]
            trunc_LB=LB[idx_sort]
            trunc_dot_prod=self.distance_[idx].dot_product_[idx_sort]

            listDP.append(DP(idx_sort, trunc_dist, trunc_LB, trunc_dot_prod))

        for i in range(1,nDP):
            line=self.distance_[idx].next_line()
            non_overlap_mask = np.arange(max(0,i-self.wlens_[idx]+1), min(nDP,i+wlen))
            line[non_overlap_mask] = np.inf
            MP[i]=np.min(line)
            IP[i]=np.argmin(line)
            #we don't need the lower bound for the last index 
            if idx+1< self.wlens_.shape[0]:
                LB=self.CompLB(idx,i)
                idx_sort=np.argsort(LB)[:self.p]
                trunc_dist=line[idx_sort]
                trunc_LB=LB[idx_sort]
                trunc_dot_prod=self.distance_[idx].dot_product_[idx_sort]
                
                listDP.append(DP(idx_sort, trunc_dist, trunc_LB, trunc_dot_prod))

        return MP, IP, listDP


    def updateDistAndLB(self, idx:int, i:int, j:int,dot_product:np.ndarray, LB:np.ndarray)-> tuple:
        #should be vectorized to be more efficient
        """Update the distance and lowerbound for the sequences i and j from a length to the next one 
            Args:
                idx(int): window length index
                i(int): offset of the first subsequence (the one for which we don't know Ti,l+k)
                j(int): offset of the second subsequence (the one for which we know Tj,l+k)
                dot_product(np.ndarray): dot product QTi,j for len wlen
                LB(np.ndarray): lower bound of Di,j for len l
            Returns:
                new_distance(np.ndarray): updated Di,j for len l+k
                new_LB(np.ndarray):updated LB for len l+k
        """
        wlen=self.wlens_[idx]
        newlen=wlen+1
        new_dot_product=dot_product+self.signal_[i+newlen-1]*self.signal_[j+newlen-1]
        #we check if the increase of the length don't create an overlap
        
        if np.abs(i-j)<newlen:
            new_dist=np.inf
            new_LB=np.inf
        else:    
            new_dist=(new_dot_product-newlen*self.distance_[idx+1].means_[i]*self.distance_[idx+1].means_[j])/(newlen*self.distance_[idx+1].stds_[i]*newlen*self.distance_[idx+1].stds_[j])
            new_dist= np.sqrt(np.clip(2*(1-new_dist),0,None))
        
            new_LB= LB * self.distance_[idx].stds_[j]/self.distance_[idx+1].stds_[j]

        return new_dist, new_LB, new_dot_product
 
    def ComputeSubMP(self,idx:int)->tuple:
        """ Compute the SubMatrixProfile from the profile of len l to the profile of len l+1
            Args: 
                idx(int): window length index
            Returns:
                bBestM(Bool): indicate if the subMP is sufficient to obtain the whole MatrixProfile
                SubMP(np.ndarray): subMatrixProfile 
                IP(np.ndarray): SubIndexProfile
        """
        wlen=self.wlens_[idx]
        next_nDP=self.n-wlen
        SubMP=np.zeros(next_nDP)
        SubIP = np.zeros(next_nDP).astype(int)
        minDistAbs, minLBAbs = np.inf, np.inf
        nonValidDP=[]
        for j in range(next_nDP):
            minDist = np.inf
            DP_iter=self.listDP[j]
            #the maximum LB is at the last position
            maxLB=DP_iter.LB[-1]
            for e in range(self.p):
                i=DP_iter.idxs[e]
                if i<next_nDP:
                    #it's possible to have i>=next_nDP if we have not compute complete matrix profiles for a long time
                    dot_product=DP_iter.dot_product[e]
                    LB=DP_iter.LB[e]
                    e_dist, e_LB, e_dot_product = self.updateDistAndLB(idx,i,j,dot_product,LB)   
                    DP_iter.distance[e], DP_iter.LB[e], DP_iter.dot_product[e] = e_dist, e_LB, e_dot_product
                    minDist=min(minDist,e_dist)
                    if minDist==e_dist:
                        ind=i
            #we update DP
            self.listDP[j]=DP_iter
            if minDist<maxLB:
                minDistAbs=min(minDistAbs,minDist)
                SubMP[j]=minDist
                SubIP[j]=ind
            else:
                minLBAbs=min(minLBAbs,maxLB)
                SubMP[j]=np.inf
                nonValidDP.append([j,maxLB])
                
                
        bBestM=minDistAbs<minLBAbs
        
        if (not bBestM) and len(nonValidDP)<(next_nDP*np.log(self.p)/np.log(next_nDP)):
            for ind,maxLB in nonValidDP:
                if maxLB<minDistAbs:
                    
                    line=self.distance_[idx+1].first_line(ind)
                    non_overlap_mask = np.arange(max(0,ind-self.wlens_[idx+1]+1), min(next_nDP,ind+wlen+1))
                    line[non_overlap_mask]=np.inf
                    SubIP[ind]=np.argmin(line)
                    SubMP[ind]=np.min(line)
                    
                    if idx+2< self.wlens_.shape[0]:
                        LB=self.CompLB(idx+1,ind)
                        
                        idx_sort=np.argsort(LB)[:self.p]
                        trunc_dist=line[idx_sort]
                        trunc_LB=LB[idx_sort]
                        trunc_dot_prod=self.distance_[idx+1].dot_product_[idx_sort]
                        
                        self.listDP[ind]=(DP(idx_sort, trunc_dist, trunc_LB, trunc_dot_prod))
                    
            bBestM=True

        return bBestM,SubMP, SubIP

        
    def Valmod(self)->np.ndarray:
        """
        Compute the Variable length matrix profile (VALMP)
        Args:
            signal (np.ndarray): signal. shape (n_samples,n_dimension)

        """

        MP, IP, self.listDP = self.ComputeMatrixProfile(0)
        nDP=self.n-self.min_wlen+1
        self.VALMP = VALMP(nDP)
        self.VALMP.updateVALMP(MP,IP,self.min_wlen)

        for idx,wlen in enumerate(self.wlens_[:-1]):
            new_nDP= self.n-wlen
            self.VALMP.nDP=new_nDP
            bBestM, SubMP, SubIP = self.ComputeSubMP(idx)
            if bBestM:
                self.VALMP.updateVALMP(SubMP,SubIP,wlen+1)
            else:
                MP, IP, self.listDP = self.ComputeMatrixProfile(idx+1)
                self.VALMP.updateVALMP(MP, IP, wlen+1)

    def fit(self,signal):
        self.signal_=signal
        self.n=self.signal_.shape[0]
        self.distance_=[]
        self.wlens_ = np.arange(self.min_wlen,self.max_wlen+1)
        for idx,wlen in enumerate(self.wlens_):
            self.distance_.append(getattr(distance,self.distance_name)(wlen,**self.distance_params))
            self.distance_[idx].fit(self.signal_)
        self.Valmod()

class VALMP(object):
    def __init__(self, nDP:int) -> None:
        """Initialization
            Args: nDP = initial number of subsequences
        """
        self.nDP = nDP
        self.distances = np.inf * np.ones(self.nDP)
        self.normDistances = np.inf * np.ones(self.nDP)
        self.lengths = np.zeros(self.nDP)
        self.indices = np.zeros(self.nDP)

    def updateVALMP(self, MPnew:np.ndarray, IP:np.ndarray, wlen:int) -> None:
        """Update of VALMP
            Args: 
                MPnew(np.ndarray): matrix profile for the current length
                IP(np.ndarray): index profile for the current length
                wlen(int): current length
                """
        lNormDist = MPnew / np.sqrt(wlen)
        for i in range(self.nDP):
            if lNormDist[i] < self.normDistances[i]:
                self.distances[i]=MPnew[i]
                self.normDistances[i]=lNormDist[i]
                self.lengths[i] = wlen
                self.indices[i] = IP[i]
    
class DP(object):
    
    def __init__(self, idxs:np.ndarray, distance:np.ndarray, LB:np.ndarray, dot_product:np.ndarray )-> None:

        self.idxs = idxs
        self.distance = distance
        self.LB = LB
        self.dot_product =dot_product
        
    


In [21]:
signal_generator = synth.SignalGenerator(2,max_rep=2,length_fluctuation=0.7)
signal,labels= signal_generator.generate()
signal_generator.plot()

In [6]:
def z_matrixprofile(signal,wlen): 
    X =sliding_window_view(signal,wlen)
    X_bar = np.mean(X,axis=1).reshape(-1,1)
    X = X-X_bar
    X = X/np.linalg.norm(X,axis=1).reshape(-1,1)
    A = cdist(X,X)
    for i in range(wlen):
        np.fill_diagonal(A[i:,:],np.inf)
        np.fill_diagonal(A[:,i:],np.inf)
    P = np.min(A,axis=1)
    IDX = np.argmin(A,axis = 1)
    return P,IDX

In [7]:
P,IDX=z_matrixprofile(signal,100)
IDX

array([259, 260, 376, 377, 344, 345, 346, 227, 228, 229, 230, 231, 232,
       233, 234, 235, 236, 237, 238, 239, 240, 361, 362, 363, 364, 365,
       366, 367, 368, 369, 247, 248, 249, 250, 251, 252, 253, 254, 255,
       256, 257, 258, 417, 418, 421, 422, 423, 424, 425, 427, 427, 428,
       429, 430, 429, 430, 430, 431, 432, 433, 433, 433, 433, 434, 435,
       436, 436, 436, 436, 437, 437, 439, 439, 439, 439, 440, 440, 440,
       440, 440, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450,
       451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463,
       464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476,
       477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502,
       503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515,
       516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528,
       529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 54

In [8]:
self = ValmodPairMotif(4,60,130,50,'UnitEuclidean')
self.fit(signal)

/var/folders/dc/2vgy793x7t15h4j_f_cy_88h0000gr/T/ipykernel_36408/587856429.py:43: RuntimeWarning:

invalid value encountered in sqrt



In [16]:
amin=np.argmin(self.VALMP.normDistances)
print(self.VALMP.indices[amin])
print(self.VALMP.lengths[amin])

464.0
97.0
